# tf.contrib.learn Quickstart

TensorFlow’s high-level machine learning API (tf.contrib.learn) makes it easy to configure, train, and evaluate a variety of machine learning models. In this quickstart tutorial, you’ll use tf.contrib.learn to construct a neural network classifier and train it on Fisher’s Iris data set to predict flower species based on sepal/petal geometry. You’ll perform the following five steps:

1. Load CSVs containing Iris training/test data into a TensorFlow Dataset
2. Construct a neural network classifier
3. Fit the model using the training data
4. Evaluate the accuracy of the model
5. Classify new samples

In [14]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

#data sets
IRIS_TRAINING = "data/iris_training.csv"
IRIS_TEST = "data/iris_test.csv"

#Load datasets
training_set = tf.contrib.learn.datasets.base.load_csv(filename = IRIS_TRAINING, 
                                                       target_dtype = np.int)
test_set = tf.contrib.learn.datasets.base.load_csv(filename = IRIS_TEST, 
                                                   target_dtype = np.int)

#Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("",dimension = 4)]

#Build 3 layer DNN with 10, 20, 10 units respectively
classifier = tf.contrib.learn.DNNClassifier(feature_columns = feature_columns,
                                           hidden_units  = [10,20,10],
                                           n_classes = 3,
                                           model_dir="/tmp/iris_model")

#Fit model
classifier.fit(x=training_set.data,
              y=training_set.target,
              steps = 2000)

#Evaluate accuracy
accuracy_score = classifier.evaluate(x=test_set.data,
                                    y=test_set.target)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

#Classify two new flower samples.
new_samples = np.array([[6.4,3.2,4.5,1.5],[5.8,3.1,5.0,1.7]],dtype = float)
y= classifier.predict(new_samples)
print('Predictions: {}'.format(str(y)))

Accuracy: 0.966667
Predictions: [1 2]


# Iris 데이터 CSV 파일을 텐서플로우로 불러오기

Iris 데이터셋은 Iris 종 내에서 서로 연관된 Iris setosa, Iris virginica, and Iris versicolor 세 종이 각각 50개씩의 표본으로 구성되어 150개 행의 데이터로 되어있습니다. 각각의 행은 각 꽃의 표본에 대한 다음의 정보를 담고 있습니다 : 꽃받침 길이, 꽃받침 너비, 꽃잎 길이, 꽃잎 너비, 그리고 꽃의 종류. 꽃의 종류는 정수로 표현
0 : Iris setosa , 1: Iris versicolor , 2: Iris virginica

이 튜토리얼을 위해서는 Iris data는 임의적으로 섞인 후에, 두 개의 따로 떨어진 CSV 파일로 나누어져야 합니다. 이는 120개의 표본을 갖는 훈련 데이터(iris_training.csv)와 30개의 표본을 갖는 테스트 데이터(iris_test.csv)입니다.

시작하기 위해선, 먼저 텐서플로우와 numpy를 불러옵니다<br>
그 다음, learn.datasets.base에 있는 load_csv() 함수를 이용하여 훈련 셋과 테스트 셋을 Dataset으로 불러옵니다. 

load_csv() 함수는 두 개의 인자를 요구합니다.

1. filename, CSV 파일이 존재하는 파일의 경로
2. target_dtype, dataset의 목표 값의 numpy 데이터형

여기에서 목표 값(모델을 훈련시켜 예측하려고 하는 값)은 0–2의 정수로 구성된 꽃의 종입니다. 따라서, 적절한 numpy 데이터형은 np.int입니다.

In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

# 데이터셋
IRIS_TRAINING = "data/iris_training.csv"
IRIS_TEST = "data/iris_test.csv"

# 데이터셋을 불러옵니다.
training_set = tf.contrib.learn.datasets.base.load_csv(filename=IRIS_TRAINING, target_dtype=np.int)
test_set = tf.contrib.learn.datasets.base.load_csv(filename=IRIS_TEST, target_dtype=np.int)

그 다음, 길이 및 너비 등의 특성 데이터와 목표 값에 변수를 할당합니다.<br> 훈련 데이터셋의 특성 데이터는 x_train, 테스트 데이터셋의 특성 데이터는 x_test, 훈련 데이터셋의 목표 값은 y_train, 테스트 데이터셋의 목표 값은 y_test입니다.<br> tf.contrib.learn의 Dataset은 named tuples이며, 순차적으로 데이터와 목표 필드(역주 : namedtuple의 field_name을 말합니다)를 통해 특성 데이터와 목표 값에 접근할 수 있습니다.

In [25]:
x_train, x_test, y_train, y_test = training_set.data, test_set.data, \
  training_set.target, test_set.target

후술할 “Iris 훈련 데이터로 DNNClassifier 피팅하기”에서, x_train과 y_train을 이용하여 모델을 훈련시키고, “모델 정확도 평가하기”에서는 x_test와 y_test를 이용할 것입니다. 하지만 먼저, 다음 문단에선 모델을 구성해봅시다.

# 딥 인공신경망 분류기 만들기

tf.contrib.learn은 데이터로 훈련과 평가를 실행할 수 있도록 곧장 사용할 수 있는, Estimator라 불리는 여러 가지의 미리 정의된 모델을 제공합니다. 여기에서는 Iris data를 피팅하기 위해 딥 인공 신경망 모델을 설정하도록 합시다. tf.contrib.learn을 이용하면, DNNClassifier를 한 줄 만에 인스턴스화할 수 있습니다.

In [17]:
# 10-20-10의 구조를 갖는 3층 DNN를 만듭니다
classifier = tf.contrib.learn.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3)

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.


위의 코드는 각각 10, 20, 10개의 뉴런으로 이루어진 3개의 은닉층을 포함한 DNNClassifier 모델을 생성합니다. 이는 (hidden_units=[10, 20, 10]), 그리고 세 개의 목표 분류 (n_classes=3)에 순차적으로 따른 것입니다.

# Iris 훈련 데이터로 DNNClassifier 피팅하기

이제 DNN classifier 모델을 설정했으니, fit 메소드를 이용하여 Iris 훈련 데이터로 이를 피팅할 수 있습니다. 특성 데이터(x_train)와 목표 값(y_train), 그리고 train할 단계 수(여기서는 200) 인자로 넘겨줍니다.

In [18]:
# Fit model
classifier.fit(x=x_train, y=y_train, steps=200)

DNNClassifier(hidden_units=[10, 20, 10], dropout=None, optimizer=None, feature_columns=None)

classifier에서 모델의 상태는 유지됩니다. 이는, 만약 원한다면 모델을 반복하여 학습시킬 수 있다는 것을 의미합니다. 예를 들어서, 위의 한 줄은 다음의 두 줄과 완벽하게 같습니다.

In [19]:
classifier.fit(x=x_train, y=y_train, steps=100)

DNNClassifier(hidden_units=[10, 20, 10], dropout=None, optimizer=None, feature_columns=None)

하지만, 만약 학습되는 동안에 모델을 추적하고 싶은 것이라면, (위와 같은 두 줄) 대신에 로그를 남기기 위해서 텐서플로우의 monitor를 사용하는 게 낫습니다.

# 모델 정확도 평가하기

이제 Iris 테스트 데이터에 맞춰 DNNClassifier 모델을 피팅했습니다. 이제, evaluate 메소드를 이용하여 Iris 테스트 데이터로 모델의 정확도를 확인해볼 수 있습니다. evaluate는 fit과 같이 특성 데이터와 목표 값을 인자로 건내받고, 평가 결과로서 dict를 반환합니다. 다음의 코드는 Iris 테스트 데이터—x_test와 y_test—를 건내받아, 결과값으로 accuracy를 출력합니다.

In [20]:
accuracy_score = classifier.evaluate(x=x_test, y=y_test)["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

Accuracy: 0.966667


# 새로운 표본 분류하기

새로운 표본을 분류하기 위해 estimator의 predict() 메소드를 이용합시다. 예를 들어, 다음의 두 가지 새로운 꽃의 표본이 있다고 해봅시다 :

In [21]:
# 새로운 두 꽃의 표본을 분류합니다.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5], [5.8, 3.1, 5.0, 1.7]], dtype=float)
y = classifier.predict(new_samples)
print ('Predictions: {}'.format(str(y)))

Predictions: [1 2]


predict() 메소드는 각 표본의 예측 결과를 하나씩 배열로 반환합니다. <br>따라서 모델은 첫 번째 표본을 Iris versicolor, 두 번째 표본을 Iris virginica로 예측하였습니다.

# 추가적인 자료

tf.contrib.learn에 대해 추가적인 참고 자료를 원한다면, 공식적인 API docs를 살펴보십시오.
선형 모델을 생성하기 위해서 tf.contrib.learn을 이용하는 것에 대해 좀 더 배우기 위해선 Large-scale Linear Models with TensorFlow를 살펴보십시오.
브라우저에서의 신경망 모델링과 시각화를 체험해보기 위해선, Deep Playground를 살펴보십시오.
신경망에 대한 좀 더 심화된 튜토리얼을 원한다면 Convolutional Neural Networks와 Recurrent Neural Networks를 살펴보십시오.